# 🔊 DEMUCS + AUDIOSR (Auto-Fix) di Google Colab
Notebook ini sudah diperbarui agar instalasi AudioSR stabil dan bebas error `git`, `setup.py`, atau koneksi.
---

In [ ]:
# 1️⃣ Persiapan environment (auto-fix instalasi)
!nvidia-smi
!pip install --upgrade pip setuptools wheel
!apt-get update -y && apt-get install -y git build-essential cmake ffmpeg unzip

# Install Torch + Demucs
!pip install torch==2.2.1 torchaudio==2.2.1 --index-url https://download.pytorch.org/whl/cu121
!pip install demucs

# Bersihkan instalasi lama AudioSR
!rm -rf AudioSR AudioSR-main audiosr.zip
!pip uninstall -y audiosr || true

import os
try:
    print('🌀 Mencoba clone dari GitHub...')
    os.system('git clone https://github.com/haoheliu/AudioSR.git')
    os.chdir('AudioSR')
except Exception:
    print('⚠️ Gagal clone, mencoba unduh ZIP...')
    os.system('wget https://github.com/haoheliu/AudioSR/archive/refs/heads/main.zip -O audiosr.zip')
    os.system('unzip -o audiosr.zip')
    os.chdir('AudioSR-main')

print('📁 Folder kerja sekarang:')
!pwd
!ls -l | head

# Instalasi AudioSR tanpa build-isolation
!pip install -e . --no-build-isolation

import torch
print('Torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())

In [ ]:
# 2️⃣ Upload file audio
from google.colab import files
from IPython.display import Audio
import torchaudio

uploaded = files.upload()
audio_path = list(uploaded.keys())[0]
print(f'File yang diupload: {audio_path}')
display(Audio(audio_path))

In [ ]:
# 3️⃣ Pisahkan vokal & instrumen (Demucs)
from demucs import pretrained
from demucs.apply import apply_model
import torch, torchaudio

model = pretrained.get_model('htdemucs')
model.to('cuda' if torch.cuda.is_available() else 'cpu')

wav, sr = torchaudio.load(audio_path)
ref = wav.mean(0)
wav = (wav - ref.mean()) / ref.std()

with torch.no_grad():
    out = apply_model(model, wav[None], device='cuda' if torch.cuda.is_available() else 'cpu')[0]

torchaudio.save('vocals.wav', out[0], sr)
torchaudio.save('instrumental.wav', out[1:].sum(0), sr)

print('✅ Vokal dan instrumen berhasil dipisahkan!')
display(Audio('vocals.wav'))

In [ ]:
# 4️⃣ Tingkatkan kualitas audio (AudioSR)
from audiosr import build_model, super_resolution
import torchaudio, torch

model_sr = build_model(model_name='basic_v1')
model_sr.to('cuda' if torch.cuda.is_available() else 'cpu')

audio, sr = torchaudio.load('vocals.wav')
audio = audio.to('cuda' if torch.cuda.is_available() else 'cpu')

audio_sr = super_resolution(model_sr, audio, sr)
torchaudio.save('vocals_enhanced.wav', audio_sr.cpu(), sr * 2)

print('✅ Audio berhasil ditingkatkan kualitasnya!')
from IPython.display import Audio as Player
display(Player('vocals_enhanced.wav'))

In [ ]:
# 5️⃣ Download hasil ke komputer
from google.colab import files
files.download('vocals.wav')
files.download('instrumental.wav')
files.download('vocals_enhanced.wav')